In [11]:
from pathlib import Path
from datetime import datetime
from os import PathLike
from typing import Union
import dateutil

# data manipulation and analysis
import numpy as np 
import pandas as pd
from astral import Astral

# Visualization
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from matplotlib.dates import date2num
import seaborn as sns
from prettytable import PrettyTable
import plotly.express as px # An interactive graphing library that makes interactive, publication-quality graphs online.
import plotly.graph_objs as go # An interactive graphing library that makes interactive, publication-quality graphs online.

# ML model selection and evaluation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

pd.set_option("display.max_rows", None, "display.max_columns", None)

In [33]:
imputed_demand_dir = '/Users/yashwanthkaruparthi/Developer/energy_demand/research/data/3-imputed/demand'
imputed_demand_path = Path(imputed_demand_dir)

imputed_weather_dir = '/Users/yashwanthkaruparthi/Developer/energy_demand/research/data/3-imputed/weather-toronto'
imputed_weather_path = Path(imputed_weather_dir)

calculated_features_dir = '/Users/yashwanthkaruparthi/Developer/energy_demand/research/data/4-calculated-features'
calculated_features_path = Path(calculated_features_dir)

interim_data_dir = '/Users/yashwanthkaruparthi/Developer/energy_demand/research/data/5-interim'
interim_data_path = Path(interim_data_dir)

clean_data_dir = '/Users/yashwanthkaruparthi/Developer/energy_demand/research/data/6-clean'
clean_data_path = Path(clean_data_dir)

### Join calendar features and imputed demand data

In [10]:

cal_df = pd.read_csv(calculated_features_path / 'calendar.csv', index_col=0, parse_dates=True,date_parser=dateutil.parser.parse)
cal_df.info()

/var/folders/n5/mlsmdgns0kqfybw1r9qm4n5w0000gp/T/ipykernel_7528/2926803345.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  cal_df = pd.read_csv(calculated_features_path / 'calendar.csv', index_col=0, parse_dates=True,date_parser=dateutil.parser.parse)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 222840 entries, 1994-01-01 00:00:00 to 2019-06-03 23:00:00
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype
---  ------           --------------   -----
 0   hour_of_day      222840 non-null  int64
 1   year             222840 non-null  int64
 2   month            222840 non-null  int64
 3   day_of_week      222840 non-null  int64
 4   day_of_year      222840 non-null  int64
 5   week_of_year     222840 non-null  int64
 6   quarter          222840 non-null  int64
 7   stat_hol         222840 non-null  bool 
 8   day_light_hours  222840 non-null  bool 
dtypes: bool(2), int64(7)
memory usage: 14.0 MB


In [12]:
power_df = pd.read_csv(imputed_demand_path / 'demand.csv', index_col=0, parse_dates=True,
                      date_parser=dateutil.parser.parse)
power_df.rename(columns={'ont_demand': 'hourly_demand'}, inplace=True)

power_df.info()

/var/folders/n5/mlsmdgns0kqfybw1r9qm4n5w0000gp/T/ipykernel_7528/1541935240.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  power_df = pd.read_csv(imputed_demand_path / 'demand.csv', index_col=0, parse_dates=True,


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 222840 entries, 1994-01-01 00:00:00 to 2019-06-03 23:00:00
Data columns (total 1 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   hourly_demand  222840 non-null  float64
dtypes: float64(1)
memory usage: 3.4 MB


In [13]:
feat_df = pd.concat([cal_df, power_df], axis=1, sort=True)
feat_df.head()

,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand
1994-01-01 00:00:00,0,1994,1,5,1,52,1,True,False,14422.0
1994-01-01 01:00:00,1,1994,1,5,1,52,1,True,False,13845.0
1994-01-01 02:00:00,2,1994,1,5,1,52,1,True,False,13372.0
1994-01-01 03:00:00,3,1994,1,5,1,52,1,True,False,13025.0
1994-01-01 04:00:00,4,1994,1,5,1,52,1,True,False,12869.0


In [14]:
feat_df.shape

(222840, 10)

In [15]:
feat_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 222840 entries, 1994-01-01 00:00:00 to 2019-06-03 23:00:00
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   hour_of_day      222840 non-null  int64  
 1   year             222840 non-null  int64  
 2   month            222840 non-null  int64  
 3   day_of_week      222840 non-null  int64  
 4   day_of_year      222840 non-null  int64  
 5   week_of_year     222840 non-null  int64  
 6   quarter          222840 non-null  int64  
 7   stat_hol         222840 non-null  bool   
 8   day_light_hours  222840 non-null  bool   
 9   hourly_demand    222840 non-null  float64
dtypes: bool(2), float64(1), int64(7)
memory usage: 15.7 MB


In [17]:
feat_df.to_csv(interim_data_path / 'demand.csv')

### Join demand, demand features, calendar features, weather features

In [19]:
int_df = pd.read_csv(interim_data_path / 'demand.csv', index_col=0, parse_dates=True,date_parser=dateutil.parser.parse)
int_df.rename(columns ={'ont_demand': 'hourly_demand'}, inplace=True)
int_df.info()

/var/folders/n5/mlsmdgns0kqfybw1r9qm4n5w0000gp/T/ipykernel_7528/2416560666.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  int_df = pd.read_csv(interim_data_path / 'demand.csv', index_col=0, parse_dates=True,date_parser=dateutil.parser.parse)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 222840 entries, 1994-01-01 00:00:00 to 2019-06-03 23:00:00
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   hour_of_day      222840 non-null  int64  
 1   year             222840 non-null  int64  
 2   month            222840 non-null  int64  
 3   day_of_week      222840 non-null  int64  
 4   day_of_year      222840 non-null  int64  
 5   week_of_year     222840 non-null  int64  
 6   quarter          222840 non-null  int64  
 7   stat_hol         222840 non-null  bool   
 8   day_light_hours  222840 non-null  bool   
 9   hourly_demand    222840 non-null  float64
dtypes: bool(2), float64(1), int64(7)
memory usage: 15.7 MB


In [20]:
int_df.head()

,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand
1994-01-01 00:00:00,0,1994,1,5,1,52,1,True,False,14422.0
1994-01-01 01:00:00,1,1994,1,5,1,52,1,True,False,13845.0
1994-01-01 02:00:00,2,1994,1,5,1,52,1,True,False,13372.0
1994-01-01 03:00:00,3,1994,1,5,1,52,1,True,False,13025.0
1994-01-01 04:00:00,4,1994,1,5,1,52,1,True,False,12869.0


In [21]:
int_df.shape

(222840, 10)

In [25]:
dtypes = {'temp': np.float64, 'dew_point_temp':np.float64, 'rel_hum':np.float64,'wind_speed': np.float64, 'visibility': np.float64, 'press': np.float64,'hmdx': np.float64, 'wind_chill': np.float64, 'weather': object}
weather_df = pd.read_csv(imputed_weather_path / "weather_toronto.csv", parse_dates=True,dtype=dtypes, index_col=0, infer_datetime_format=True)

weather_df.info()

/var/folders/n5/mlsmdgns0kqfybw1r9qm4n5w0000gp/T/ipykernel_7528/3953713310.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  weather_df = pd.read_csv(imputed_weather_path / "weather_toronto.csv", parse_dates=True,


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 582433 entries, 1953-01-01 00:00:00 to 2019-06-11 23:00:00
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   temp            582433 non-null  float64
 1   dew_point_temp  582433 non-null  float64
 2   rel_hum         582433 non-null  float64
 3   wind_speed      582433 non-null  float64
 4   visibility      582433 non-null  float64
 5   press           582433 non-null  float64
 6   hmdx            77405 non-null   float64
 7   wind_chill      143782 non-null  float64
 8   weather         552686 non-null  object 
dtypes: float64(8), object(1)
memory usage: 44.4+ MB


In [31]:
weather_df.head()

,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,hmdx,wind_chill,weather
1953-01-01 00:00:00,-5.6,-6.7,93.0,10.0,25.0,99.44,NaN,-10.0,Cloudy
1953-01-01 01:00:00,-5.6,-9.4,71.0,3.0,25.0,99.46,NaN,-7.0,Cloudy
1953-01-01 02:00:00,-5.0,-8.9,72.0,0.0,19.3,99.43,NaN,NaN,Cloudy
1953-01-01 03:00:00,-5.0,-7.8,79.0,0.0,19.3,99.56,NaN,NaN,Cloudy
1953-01-01 04:00:00,-4.4,-7.2,80.0,0.0,19.3,99.55,NaN,NaN,Cloudy


In [26]:
weather_df.shape

(582433, 9)

In [27]:
clean_df = pd.concat([weather_df, int_df], axis=1, sort=True)
clean_df.head()

,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,hmdx,wind_chill,weather,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand
1953-01-01 00:00:00,-5.6,-6.7,93.0,10.0,25.0,99.44,NaN,-10.0,Cloudy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953-01-01 01:00:00,-5.6,-9.4,71.0,3.0,25.0,99.46,NaN,-7.0,Cloudy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953-01-01 02:00:00,-5.0,-8.9,72.0,0.0,19.3,99.43,NaN,NaN,Cloudy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953-01-01 03:00:00,-5.0,-7.8,79.0,0.0,19.3,99.56,NaN,NaN,Cloudy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953-01-01 04:00:00,-4.4,-7.2,80.0,0.0,19.3,99.55,NaN,NaN,Cloudy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
clean_df.shape

(582433, 19)

In [29]:
clean_df.isnull().sum()

temp                    0
dew_point_temp          0
rel_hum                 0
wind_speed              0
visibility              0
press                   0
hmdx               505028
wind_chill         438651
weather             29747
hour_of_day        359593
year               359593
month              359593
day_of_week        359593
day_of_year        359593
week_of_year       359593
quarter            359593
stat_hol           359593
day_light_hours    359593
hourly_demand      359593
dtype: int64

In [32]:
# Create a daily_peak column
# Peak Daily demand
clean_df = clean_df.join(feat_df.groupby(by=['year', 'day_of_year'])['hourly_demand'].max(),on=['year', 'day_of_year'], rsuffix='_peak_in_day_in_year')
clean_df.rename(columns={'hourly_demand_peak_in_day_in_year':'daily_peak'}, inplace=True)

In [34]:
clean_df.to_csv(clean_data_path / 'oversized.csv')

In [36]:
dtypes = {'temp': np.float64, 'dew_point_temp':np.float64, 'rel_hum':np.float64,'wind_speed': np.float64, 'visibility': np.float64, 'press': np.float64,'hmdx': np.float64, 'wind_chill': np.float64, 'weather': object,'hour_of_day': np.float64, 'year': np.float64, 'month': np.float64,'day_of_week': np.float64, 'day_of_year': np.float64, 'week_of_year': np.float64,'quarter': np.float64, 'stat_hol': object, 'day_light_hours': object,'hourly_demand': np.float64, 'daily_peak': np.float64, 'weekly_peak': np.float64,'monthly_peak': np.float64, 'quarterly_peak': np.float64, 'annual_peak': np.float64,'hour_as_pct_of_daily_peak': np.float64,'daily_peak_as_pct_of_weekly_peak': np.float64,'monthly_peak_as_pct_of_annual_peak': np.float64,
'quarterly_peak_as_pct_of_annual_peak': np.float64, 'daily_peak_as_pct_of_annual_peak': np.float64}

oversized_df = pd.read_csv(clean_data_path / "oversized.csv", parse_dates=True,dtype=dtypes, index_col=0, infer_datetime_format=True)

oversized_df.info()

/var/folders/n5/mlsmdgns0kqfybw1r9qm4n5w0000gp/T/ipykernel_7528/228787538.py:4: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  oversized_df = pd.read_csv(clean_data_path / "oversized.csv", parse_dates=True,dtype=dtypes, index_col=0, infer_datetime_format=True)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 582433 entries, 1953-01-01 00:00:00 to 2019-06-11 23:00:00
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   temp             582433 non-null  float64
 1   dew_point_temp   582433 non-null  float64
 2   rel_hum          582433 non-null  float64
 3   wind_speed       582433 non-null  float64
 4   visibility       582433 non-null  float64
 5   press            582433 non-null  float64
 6   hmdx             77405 non-null   float64
 7   wind_chill       143782 non-null  float64
 8   weather          552686 non-null  object 
 9   hour_of_day      222840 non-null  float64
 10  year             222840 non-null  float64
 11  month            222840 non-null  float64
 12  day_of_week      222840 non-null  float64
 13  day_of_year      222840 non-null  float64
 14  week_of_year     222840 non-null  float64
 15  quarter          222840 non-null  float64
 16  stat

In [37]:
clean_df = oversized_df.dropna(subset=['hourly_demand'])

clean_df.to_csv(clean_data_path / "clean.csv")

In [43]:
clean_df.tail()

,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,hmdx,wind_chill,weather,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand,daily_peak
2019-06-03 19:00:00,14.3,0.5,39.0,28.0,24.1,99.68,NaN,NaN,Mainly Clear,19.0,2019.0,6.0,0.0,154.0,23.0,2.0,False,True,15518.0,15518.0
2019-06-03 20:00:00,12.7,2.2,49.0,23.0,24.1,99.74,NaN,NaN,NaN,20.0,2019.0,6.0,0.0,154.0,23.0,2.0,False,True,15513.0,15518.0
2019-06-03 21:00:00,11.1,3.0,57.0,14.0,24.1,99.81,NaN,NaN,NaN,21.0,2019.0,6.0,0.0,154.0,23.0,2.0,False,False,14664.0,15518.0
2019-06-03 22:00:00,11.6,2.3,53.0,17.0,24.1,99.81,NaN,NaN,Mainly Clear,22.0,2019.0,6.0,0.0,154.0,23.0,2.0,False,False,13517.0,15518.0
2019-06-03 23:00:00,8.5,2.7,67.0,7.0,24.1,99.80,NaN,NaN,NaN,23.0,2019.0,6.0,0.0,154.0,23.0,2.0,False,False,12464.0,15518.0


In [39]:
clean_df.isnull().sum()

temp                    0
dew_point_temp          0
rel_hum                 0
wind_speed              0
visibility              0
press                   0
hmdx               188759
wind_chill         172387
weather             29625
hour_of_day             0
year                    0
month                   0
day_of_week             0
day_of_year             0
week_of_year            0
quarter                 0
stat_hol                0
day_light_hours         0
hourly_demand           0
daily_peak              0
dtype: int64

In [40]:
dtypes = {'temp': np.float64, 'dew_point_temp':np.float64, 'rel_hum':np.float64,'wind_speed': np.float64, 'visibility': np.float64, 'press': np.float64,'hmdx': np.float64, 'wind_chill': np.float64, 'weather': object,'hour_of_day': np.float64, 'year': np.float64, 'month': np.float64,'week_of_year': np.float64,'quarter': np.float64, 'stat_hol': object, 'day_light_hours': object,'weekly_peak': np.float64,'annual_peak': np.float64,'daily_peak_as_pct_of_weekly_peak': np.float64,'monthly_peak_as_pct_of_annual_peak': np.float64,'daily_peak_as_pct_of_annual_peak': np.float64}

df = pd.read_csv(clean_data_path / "clean.csv", parse_dates=True, dtype=dtypes, index_col=0, infer_datetime_format=True)

/var/folders/n5/mlsmdgns0kqfybw1r9qm4n5w0000gp/T/ipykernel_7528/1587288919.py:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(clean_data_path / "clean.csv", parse_dates=True, dtype=dtypes, index_col=0, infer_datetime_format=True)


In [42]:
df.head()

,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,hmdx,wind_chill,weather,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand,daily_peak
1994-01-01 00:00:00,-1.2,-3.8,83.0,15.0,19.3,99.91,NaN,-6.0,Cloudy,0.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,14422.0,16892.0
1994-01-01 01:00:00,-0.9,-3.0,86.0,20.0,16.1,99.91,NaN,-6.0,Cloudy,1.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,13845.0,16892.0
1994-01-01 02:00:00,-0.7,-3.2,83.0,15.0,16.1,99.87,NaN,-5.0,Cloudy,2.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,13372.0,16892.0
1994-01-01 03:00:00,-0.8,-2.4,89.0,15.0,12.9,99.81,NaN,-5.0,Cloudy,3.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,13025.0,16892.0
1994-01-01 04:00:00,-1.0,-3.3,84.0,19.0,16.1,99.77,NaN,-6.0,Mostly Cloudy,4.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,12869.0,16892.0


In [44]:
clean_cut_df = clean_df.loc['1994': '2018']
clean_cut_df.to_csv(clean_data_path / "clean-cut.csv")

In [46]:
dtypes = {'temp': np.float64, 'dew_point_temp':np.float64, 'rel_hum':np.float64,
          'wind_speed': np.float64, 'visibility': np.float64, 'press': np.float64,
          'hmdx': np.float64, 'wind_chill': np.float64, 'weather': object,
         'hour_of_day': np.float64, 'year': np.float64, 'month': np.float64,
         'day_of_week': np.float64, 'day_of_year': np.float64, 'week_of_year': np.float64,
          'quarter': np.float64, 'stat_hol': object, 'day_light_hours': object,
          'hourly_demand': np.float64, 'daily_peak': np.float64, 'weekly_peak': np.float64,
          'monthly_peak': np.float64, 'quarterly_peak': np.float64, 'annual_peak': np.float64,
          'hour_as_pct_of_daily_peak': np.float64, 'daily_peak_as_pct_of_weekly_peak': np.float64,
         'monthly_peak_as_pct_of_annual_peak': np.float64,
          'quarterly_peak_as_pct_of_annual_peak': np.float64, 'daily_peak_as_pct_of_annual_peak': np.float64}

clean_cut_df = pd.read_csv(clean_data_path / "clean-cut.csv", parse_dates=True,
                 dtype=dtypes, index_col=0, infer_datetime_format=True)

clean_cut_df.info()

/var/folders/n5/mlsmdgns0kqfybw1r9qm4n5w0000gp/T/ipykernel_7528/418193300.py:13: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  clean_cut_df = pd.read_csv(clean_data_path / "clean-cut.csv", parse_dates=True,


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 219144 entries, 1994-01-01 00:00:00 to 2018-12-31 23:00:00
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   temp             219144 non-null  float64
 1   dew_point_temp   219144 non-null  float64
 2   rel_hum          219144 non-null  float64
 3   wind_speed       219144 non-null  float64
 4   visibility       219144 non-null  float64
 5   press            219144 non-null  float64
 6   hmdx             34068 non-null   float64
 7   wind_chill       48918 non-null   float64
 8   weather          191347 non-null  object 
 9   hour_of_day      219144 non-null  float64
 10  year             219144 non-null  float64
 11  month            219144 non-null  float64
 12  day_of_week      219144 non-null  float64
 13  day_of_year      219144 non-null  float64
 14  week_of_year     219144 non-null  float64
 15  quarter          219144 non-null  float64
 16  stat

In [47]:
clean_cut_df.head()

,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,hmdx,wind_chill,weather,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand,daily_peak
1994-01-01 00:00:00,-1.2,-3.8,83.0,15.0,19.3,99.91,NaN,-6.0,Cloudy,0.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,14422.0,16892.0
1994-01-01 01:00:00,-0.9,-3.0,86.0,20.0,16.1,99.91,NaN,-6.0,Cloudy,1.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,13845.0,16892.0
1994-01-01 02:00:00,-0.7,-3.2,83.0,15.0,16.1,99.87,NaN,-5.0,Cloudy,2.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,13372.0,16892.0
1994-01-01 03:00:00,-0.8,-2.4,89.0,15.0,12.9,99.81,NaN,-5.0,Cloudy,3.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,13025.0,16892.0
1994-01-01 04:00:00,-1.0,-3.3,84.0,19.0,16.1,99.77,NaN,-6.0,Mostly Cloudy,4.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,12869.0,16892.0


In [48]:
clean_cut_df.tail()

,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,hmdx,wind_chill,weather,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand,daily_peak
2018-12-31 19:00:00,2.5,1.0,90.0,19.0,9.7,98.51,NaN,NaN,"Rain,Fog",19.0,2018.0,12.0,0.0,365.0,1.0,4.0,False,False,16195.0,17125.0
2018-12-31 20:00:00,2.9,1.6,92.0,21.0,9.7,98.17,NaN,NaN,"Rain,Fog",20.0,2018.0,12.0,0.0,365.0,1.0,4.0,False,False,15668.0,17125.0
2018-12-31 21:00:00,3.7,2.6,93.0,21.0,9.7,97.98,NaN,NaN,"Rain,Fog",21.0,2018.0,12.0,0.0,365.0,1.0,4.0,False,False,14987.0,17125.0
2018-12-31 22:00:00,4.0,3.0,93.0,18.0,6.4,97.61,NaN,NaN,"Rain,Fog",22.0,2018.0,12.0,0.0,365.0,1.0,4.0,False,False,14560.0,17125.0
2018-12-31 23:00:00,4.3,3.4,94.0,12.0,9.7,97.37,NaN,NaN,"Rain,Fog",23.0,2018.0,12.0,0.0,365.0,1.0,4.0,False,False,13828.0,17125.0


In [49]:
clean_cut_df.isnull().sum()

temp                    0
dew_point_temp          0
rel_hum                 0
wind_speed              0
visibility              0
press                   0
hmdx               185076
wind_chill         170226
weather             27797
hour_of_day             0
year                    0
month                   0
day_of_week             0
day_of_year             0
week_of_year            0
quarter                 0
stat_hol                0
day_light_hours         0
hourly_demand           0
daily_peak              0
dtype: int64

In [50]:
clean_cut_df.shape

(219144, 20)

In [54]:
clean_cut_df['weather'].nunique()

157